### Obtendo Dados Paginados e Salvando em um CSV

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Firefox()

In [7]:
driver.get('http://localhost:8000/#/exemplo/8')

In [13]:
from dataclasses import dataclass

@dataclass
class Usuario:
    avatar: str
    nome: str
    profissao: str
    email: str
    perfil: str
    estado: str

In [16]:
def raspar_dados_da_tabela():
    linhas = driver.find_elements(By.CSS_SELECTOR, 'table#tabela-usuarios tbody tr')
    
    registros = []
    
    for linha in linhas:
        dados = linha.get_property('children')
        
        registro = Usuario(
            avatar = dados[0].find_element(By.TAG_NAME, 'img').get_attribute('src'),
            nome = dados[1].text,
            profissao = dados[2].text,
            email = dados[3].text,
            perfil = dados[4].text,
            estado = dados[5].text
        )
        
        registros.append(registro)
    
    return registros

In [21]:
dados_da_tabela = []

while True:
    dados = raspar_dados_da_tabela()
    dados_da_tabela.extend(dados)
    
    try:
        btn_proximo = driver.find_element(By.XPATH, '//main//button[contains(text(), "Próxima")]')
        btn_proximo.click()
        
    except Exception as e:
        break

In [24]:
import csv
from dataclasses import fields
from dataclasses import asdict

with open('dados_aula6.csv', 'w', encoding='utf-8') as file:
    headers = [field.name for field in fields(Usuario)]
    file = csv.DictWriter(file, fieldnames=headers)
    
    file.writeheader()
    
    data_formatted = [asdict(data) for data in dados_da_tabela]
    file.writerows(data_formatted)